In [1]:
import chromadb
import polars as pl
from dotenv import load_dotenv
import os

load_dotenv()

articles = pl.read_csv('Articles.csv',  encoding='ISO-8859-1').with_row_index(offset = 1)

articles.head()

chroma_client = chromadb.Client()

articles = articles[:50]


In [2]:
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

model = HuggingFaceInferenceAPIEmbeddings(
    model_name='sentence-transformers/all-MiniLM-L6-v2',
    api_key= os.getenv('api_key')
)

list_articles = articles['Article'].to_list()

vectors =model.embed_documents(list_articles)
ids = [f"id{x}" for x in articles['index'].to_list()]


In [3]:
collection = chroma_client.get_or_create_collection(name='articles')

collection.add(
    documents=list_articles,
    ids=ids,
    embeddings=vectors,
)


In [4]:
query = "who is the Federal Finance Minister of Pakistan"

query_emb = model.embed_query(query)

context = collection.query(
    query_embeddings=query_emb,
    n_results=5
)

context

{'ids': [['id23', 'id27', 'id34', 'id44', 'id22']],
 'embeddings': None,
 'documents': [['ISLAMABAD: Pakistan\'s ongoing fuel shortage that has led to worsening power blackouts is weighing on its credit worthiness and hindering its ability to meet key reform targets laid out by the IMF, ratings agency Moody´s warned Monday. The country is currently in the grip of one of its worst power crises in years due to a shortfall in imported oil, with the situation exacerbated Sunday by an attack on a key powerline in restive Balochistan province. Moody´s said that increasing energy imports without addressing structural issues that create so-called circular debt "will further strain Pakistan\'s budget and balance of payments, a credit negative". "Fuel shortages also reflect the strained finances of state-owned distribution companies and the fuel importer, Pakistan State Oil corporation, and are a setback to the sector\'s progress on reforms made so far under Pakistan\'s financial support program

In [7]:
import os
from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),  # This is the default and can be omitted
)

chat_completion = client.chat.completions.create( 
    messages=[
        {
            "role" : "system",
            "content": context,
        },
        {
            "role": "user",
            "content" : query
        }
    ],
    model="llama3-8b-8192",
)
   
print(chat_completion.choices[0].message.content)

BadRequestError: Error code: 400 - {'error': {'message': "'messages.0' : for 'role:system' the following must be satisfied[('messages.0.content' : value must be a string)]", 'type': 'invalid_request_error'}}